In [7]:
!pip install google-api-python-client

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB 435.7 kB/s eta 0:00:28
   ---------------------------------------- 0.1/12.0 MB 465.5 kB/s eta 0:00:26
   ---------------------------------------- 0.1/12.0 MB 595.3 kB/s eta 0:00:20
   ---------------------------------------- 0.1/12.0 MB 607.9 kB/s eta 0:00:20
    --------------------------------------- 0.2/12.0 MB 653.6 kB/s eta 0:00:18
    --------------------------------------- 0.3/12.0 MB 682.7 kB/s eta 0:00:18
   - -------------------------------------- 0.3/12.0 MB 729.6 kB/s eta 0:00:16
   - -------------------------------------- 0.4/12.0 MB 739.0 kB/s eta 0:00:16
   - -------------------------------------- 0.4/12.0 MB 786.3 kB/s eta 0:00:15
   - -------------------------------------- 0.5/12.0 MB 805.8 kB/s eta 0:00:15


In [8]:
!pip install google-auth-oauthlib

In [9]:
!pip install google-auth-httplib2

In [10]:
!pip install beautifulsoup4

In [56]:
# https://python.langchain.com/en/latest/modules/agents/toolkits/examples/gmail.html

# Step 1: Create a json file and download the same in the working folder: https://developers.google.com/gmail/api/quickstart/python#authorize_credentials_for_a_desktop_application

# Step 2: pip install google-api-python-client, google-auth-oauthlib, google-auth-httplib2, beautifulsoup4, langchain

# Step 3: Add Users for testing this app In Authconsent Screen
# Issues resolved: https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc

# Step 4: Enable it by visiting https://console.developers.google.com/apis/api/gmail.googleapis.com/overview?project=26982114008



from langchain.agents.agent_toolkits import GmailToolkit
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
toolkit = GmailToolkit() 

# tools = toolkit.get_tools()
# print(tools)

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
os.environ['OPENAI_API_KEY'] = os.environ["OPENAI_API_KEY"]

from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0)

agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)


# agent.run("Create a gmail draft for me to editor of a letter from the perspective of a sentient parrot"
#           " who is looking to collaborate on some research with her"
#           " estranged friend, a cat. Under no circumstances may you send the message, however.")

email = agent.run("Could you search in my inbox for the email on product query and provide me the complete mail content with sender email, subject and body in json format?")
# chat_messages=[
#     SystemMessage(content='You are an expert assistant with expertize in reading mails'),
#     HumanMessage(content=f'Please provide the complete email content including the body of the mail and do not add or make up anything and return me a json with sender name, email, subject, body:\n TEXT: {email}')
# ]

# # llm=ChatOpenAI(model_name='gpt-3.5-turbo')
# email_json = llm(chat_messages).content


In [57]:
email

{'id': '18d210698d94d716',
 'threadId': '18d210644969fb25',
 'snippet': 'How long does it take to charge the PowerGuard Elite battery fully?',
 'body': 'How long does it take to charge the PowerGuard Elite battery fully?\r\n',
 'subject': 'query on product inverter battery',
 'sender': 'AKSHAT PANT <akshat.pant06@gmail.com>'}

In [66]:
product_name = agent.run(f"Identify the Product Name and what is the product category from this context and return a json: {email['body']}")

In [67]:
product_name

{'product_name': 'PowerGuard Elite battery',
 'product_category': 'inverter battery'}

In [68]:
from PyPDF2 import PdfReader
pdfreader = PdfReader('inv_battery.pdf')

text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [72]:
chat_messages = [
    SystemMessage(content='You are an expert assistant with expertise in reading user manuals of products'),
    HumanMessage(content=f'Please provide an answer to the following question {email["body"]} from the:\n TEXT: {text}')
]

llm=ChatOpenAI(model_name='gpt-3.5-turbo')

In [71]:
email['body']

'How long does it take to charge the PowerGuard Elite battery fully?\r\n'

In [74]:
query_response = llm(chat_messages).content

In [75]:
query_response

'According to the provided user manual, the PowerGuard Elite battery takes approximately 8-10 hours to charge fully. It is recommended to charge the battery for this duration before initial use to ensure optimal performance.'

In [76]:
email

{'id': '18d210698d94d716',
 'threadId': '18d210644969fb25',
 'snippet': 'How long does it take to charge the PowerGuard Elite battery fully?',
 'body': 'How long does it take to charge the PowerGuard Elite battery fully?\r\n',
 'subject': 'query on product inverter battery',
 'sender': 'AKSHAT PANT <akshat.pant06@gmail.com>'}

In [79]:
agent.run(f"Send a gmail reply to {email['sender']} for {email['subject']} with response {query_response}")

'Email sent successfully.'